# Model Template

--Please replace this Markdown cell with a description of your trading model.--

This is a template of a notebook we can use to build a trading model. Some code cells can be (should be) copied over to the trading algorithm with which you may perform backtesting.

## 1. Imports and Settings

In the code cell below, switch the production/development commented sections as necessary, and then copy the code into the algorithm:

In [ ]:
# Note: Copy to the algorithm

# Module Imports
# --------------------
import quantopian.optimize as opt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.filters import QTradableStocksUS

import numpy as np
import pandas as pd
from datetime import datetime

# Environment Settings
# --------------------
## Production 
# universe = QTradableStocksUS()
# mask = {'mask': universe}

## Development
universe = QTradableStocksUS()
mask = {'mask': universe}


# Global Configuration
# --------------------

Do not copy this code cell into the algorithm:

In [ ]:
from quantopian.research import run_pipeline

# Leave at least 6 months holding period.

## Production 
# start_date = datetime.strptime('01/01/2018', '%m/%d/%Y')
# end_date = datetime.strptime('06/01/2019', '%m/%d/%Y')

## Development
start_date = datetime.strptime('03/01/2020', '%m/%d/%Y')
end_date = datetime.strptime('03/02/2020', '%m/%d/%Y')

## 2. Helpers

The code below can be copied directly into the algorithm.

In [ ]:
# Note: Copy to the algorithm


In [ ]:
# Assertion code, if needed. Use to test out helper functions.
# Here is an example of normalize() function's assertion.

def assert_normalize():
    a = normalize(np.array([-300, -200, -100, 0, 100, 200, 300]))
    b = normalize(np.array([-200, -100, 0, 100, 200, 300, 400]))
    c = normalize(np.array([-3, -2, -1, 0, 1, 2, 3]))
    d = normalize(np.array([1,1,1,1,1]))
    assert type(a) == pd.core.series.Series, "Return type must be a pandas.core.series.Series object"
    assert (a == b).all(), "Demean should adjust the mean properly"
    assert (a == c).all(), "Incorrect normalization result"
    assert (d.sum() == 0), "If an array has the same value for all elements, normalized value should be zero for all"
    return True
assert_normalize()

## 3. Build Pipeline

The code below can be copied directly into the algorithm.

In [ ]:
# Note: Copy to the algorithm

def make_alpha_factors():
    factors = {}
    # Todo: Create factors here. One of the factors must be named `a_combined`.
    factors['a_combined'] = ...
    
    return factors
                                        

def make_pipeline():
    alpha_factors = make_alpha_factors()
    factors = {a: alpha_factors[a] for a in alpha_factors}
    pipe = Pipeline(columns=factors, screen=universe)
    
    return pipe

## 4. Run Pipeline

In [ ]:
pipe = make_pipeline()
mdf = run_pipeline(pipe, start_date, end_date).dropna(how='all')
mdf.head(5)

## 5. Analyze Pipeline Result

### 5.1. Validating the normalization process

Checking combined alpha of the first date. Should be close to 1.

In [ ]:
first_available_date = mdf.index.get_level_values(0)[0]
selector = mdf.index.get_level_values(0) == first_available_date
mdf.loc[selector]['a_combined'].abs().sum()

### 5.2. Distribution of values

More spread, the better.

In [ ]:
mdf.hist(bins=100);

### 5.3. Correlation between factors

Correlation close to 1.0 (or -1.0) means the factor is probably not needed.

In [ ]:
mdf.fillna(0).corr()  # Filling NaNs with 0 assumes empty values are the mean (z-score/rank of 0)
# mdf.corr()  # Drops NaNs; results are not much different

### 5.4. Number of stocks to trade

1. Total number of stocks considered for trading throughout all dates:

In [ ]:
mdf.index.get_level_values(1).unique().shape

2. How many stocks on average do we trade each day?

In [ ]:
mdf.dropna().groupby(level=0).agg('count').mean()[0]

## 6. Get Alpha Factors
 
We have a function below to extract alpha factors from the `factors` DataFrame we have created above. It does two things by default:

1. Replace `np.inf` and `np.nan` to 0, and
2. get the `factors['a_combined']` Series.

Although you may technically combine the alphas in this function, it is preferable to do the alpha combination step on the "Build Pipeline" step above. The reasoning:

The algorithm environment requires this function to accept a $1 \times s$ DataFrame where $s$ is the number of assets, while this research environment requires a $d \times s$ DataFrame where $d$ is the number of dates. Due to this, performing complex operations (such as getting only the extreme values of the combined alpha) may take a very long time on the Notebook environment.

In [ ]:
# Note: Adjust as necessary. In here, columns that begins with 'a_' would be
#       considered alpha factors. The rest are for analysis purposes.

filter_col = [col for col in mdf if col.startswith('a_')]
alphas = mdf.loc[:, filter_col]

# Note: Copy to the algorithm

def get_alpha(factors):
    # Replace infs and NaNs
    factors[np.isinf(factors)] = np.nan
    factors.fillna(0, inplace=True)

    combined_alpha = factors['a_combined']
    return combined_alpha

The trading algorithm will use the `combined_alpha` DataFrame.

In [ ]:
combined_alpha = get_alpha(alphas)

## 7. Analyze Alphas

### 7.1. Alphas' Statistics

Present some statistics of the alphas here.

In [ ]:
alphas.describe()

How many % of our signals are shorts?

In [ ]:
median_daily_signals = alphas.groupby(level=0).count().median()
percent_short = float(combined_alpha[combined_alpha < 0.0].count()) \
                / float(combined_alpha.count())*100

print("Median number of daily signals: {}".format(median_daily_signals))
print("% of short signals: {}%".format(percent_short))

### 7.2. Alphalens Analysis

`ANALYZE_ALL` settings:

1. `True`: Analyze all alpha factors in variable `alphas` and the final `combined_alpha` variable.
2. `False`: Analyze only the final factor i.e. the `combined_alpha` variable.

In [ ]:
ANALYZE_ALL = True

if ANALYZE_ALL:
    alphas_view = alphas.copy()
else:
    alphas_view = pd.DataFrame({'aa_combined': combined_alpha})
alphas_view.head(5)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 1 day, 1 week, 1 month, 1 quarter
periods = [1, 5, 22, 64]

# Get pricing data (extends 6 months to minimize dropping in Alphalens)
new_start_date = start_date - relativedelta(months=6)
new_end_date = end_date + relativedelta(months=6)
assets = alphas_view.reset_index()['level_1'].unique()
dates = alphas_view.reset_index()['level_0'].unique()
prices = get_pricing(assets, start_date=new_start_date, end_date=new_end_date, fields='close_price')
prices.head(5)

In [ ]:
import alphalens as al
from scipy import stats

def get_table(ic_data, ab_data):
    summary_table = pd.DataFrame()
    summary_table["Ann. Alpha"] = ab_data.loc['Ann. alpha']
    summary_table["beta"] = ab_data.loc['beta']
    summary_table["IC Mean"] = ic_data.mean()
    summary_table["IC Std."] = ic_data.std()
    summary_table["Risk-Adjusted IC"] = \
        ic_data.mean() / ic_data.std()
    t_stat, p_value = stats.ttest_1samp(ic_data, 0)
    summary_table["p-value(IC)"] = p_value

    return summary_table.apply(lambda x: x.round(3)).T


results = None
for i, col in enumerate(sorted(alphas_view.columns)):
    if i > 0:
        print('')
    print(col)
    
    # Get the factor data
    data = alphas_view[col]
    data = data[data != 0].dropna()
#     try:
    factor_data = al.utils.get_clean_factor_and_forward_returns(data,
                                                                prices,
                                                                quantiles=5,
                                                                periods=periods,
                                                                max_loss=1.
                                                               )

    # Output the results
    ic = al.performance.factor_information_coefficient(factor_data)
    ic.columns = pd.MultiIndex.from_product([[col], ic.columns])

    returns = al.performance.factor_returns(factor_data)
    ab = al.performance.factor_alpha_beta(factor_data, returns=returns)
    ab.columns = pd.MultiIndex.from_product([[col], ab.columns])

    table = get_table(ic, ab)

    if results is None:
        results = table
    else:
        results = pd.concat([results, table], axis=1)
            
#     except Exception as e:
#         print('Error: {}'.format(e))
#         continue
        
temp = None
i = 0
unique_vals = results.columns.get_level_values(0).unique()
for j, factor in enumerate(sorted(unique_vals)):
    i += 1
    res = results.xs(factor, axis=1, level=0, drop_level=False)
    
    if temp is None:
        temp = res
    else:
        temp = pd.concat([temp, res], axis=1)
        
    if i > 4 or j == len(unique_vals) - 1:
        display(temp)
        temp = None
        i = 0

## 8. Tear Sheet Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import empyrical as ep
import alphalens as al
import pyfolio as pf

from quantopian.research.experimental import get_factor_returns, get_factor_loadings

MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}

# Load risk factor loadings and returns
factor_loadings = get_factor_loadings(assets, start_date, new_end_date)
factor_returns = get_factor_returns(start_date, new_end_date)

# Fix a bug in the risk returns
factor_returns.loc[factor_returns.value.idxmax(), 'value'] = 0

def calc_perf_attrib(portfolio_returns, portfolio_pos, factor_returns, factor_loadings):
    start = portfolio_returns.index[0]
    end = portfolio_returns.index[-1]
    factor_loadings.index = factor_loadings.index.set_names(['dt', 'ticker'])
    portfolio_pos.index = portfolio_pos.index.set_names(['dt'])
    
    portfolio_pos = portfolio_pos.drop('cash', axis=1)
    portfolio_pos.columns.name = 'ticker'
    portfolio_pos.columns = portfolio_pos.columns.astype('int')
    
    return ep.perf_attrib(
        portfolio_returns, 
        portfolio_pos.stack().dropna(),
        factor_returns.loc[start:end], 
        factor_loadings.loc[start:end])

def plot_exposures(risk_exposures, ax=None):
    rep = risk_exposures.stack().reset_index()
    rep.columns = ['dt', 'factor', 'exposure']
    sns.boxplot(x='exposure', y='factor', data=rep, orient='h', ax=ax, order=risk_exposures.columns[::-1])

def compute_turnover(df):
    return df.dropna().unstack().dropna(how='all').fillna(0).diff().abs().sum(1)

def get_max_median_position_concentration(expos):
    longs = expos.loc[expos > 0]
    shorts = expos.loc[expos < 0]

    return expos.groupby(level=0).quantile([.05, .25, .5, .75, .95]).unstack()

def compute_factor_stats(factor, pricing, factor_returns,
                         factor_loadings, periods=range(1, 15),
                         view=None):
    factor_data_total = al.utils.get_clean_factor_and_forward_returns(
        factor, 
        pricing,
        quantiles=None,
        bins=(-np.inf, 0, np.inf),
        periods=periods,
        cumulative_returns=False
    )

    portfolio_returns_total = al.performance.factor_returns(factor_data_total)
    portfolio_returns_total.columns = portfolio_returns_total.columns.map(lambda x: int(x[:-1]))
    for i in portfolio_returns_total.columns:
        portfolio_returns_total[i] = portfolio_returns_total[i].shift(i)

    portfolio_returns_specific = pd.DataFrame(columns=portfolio_returns_total.columns, index=portfolio_returns_total.index)
    
    # closure
    def calc_perf_attrib_c(i, portfolio_returns_total=portfolio_returns_total, 
                           factor_data_total=factor_data_total, factor_returns=factor_returns, 
                           factor_loadings=factor_loadings):
        return calc_perf_attrib(portfolio_returns_total[i], 
                                factor_data_total['factor'].unstack().assign(cash=0).shift(i), 
                                factor_returns, factor_loadings)
    
    if view is None:
        perf_attrib = map(calc_perf_attrib_c, portfolio_returns_total.columns)
    else:
        perf_attrib = view.map_sync(calc_perf_attrib_c, portfolio_returns_total.columns)
        
    for i, pa in enumerate(perf_attrib):
        if i == 0:
            risk_exposures_portfolio = pa[0]
            perf_attribution = pa[1]
        portfolio_returns_specific[i + 1] = pa[1]['specific_returns']
    
    delay_sharpes_total = portfolio_returns_total.apply(ep.sharpe_ratio)
    delay_sharpes_specific = portfolio_returns_specific.apply(ep.sharpe_ratio)
    
    turnover = compute_turnover(factor)
    n_holdings = factor.groupby(level=0).count()
    perc_holdings = get_max_median_position_concentration(factor)
    
    return {'factor_data_total': factor_data_total, 
            'portfolio_returns_total': portfolio_returns_total,
            'portfolio_returns_specific': portfolio_returns_specific,
            'risk_exposures_portfolio': risk_exposures_portfolio,
            'perf_attribution': perf_attribution,
            'delay_sharpes_total': delay_sharpes_total,
            'delay_sharpes_specific': delay_sharpes_specific,
            'turnover': turnover,
            'n_holdings': n_holdings,
            'perc_holdings': perc_holdings,
    }

def plot_overview_tear_sheet(factor, pricing, factor_returns, factor_loadings,
                             periods=range(1, 15), view=None):
    fig = plt.figure(figsize=(16, 16))
    gs = plt.GridSpec(5, 4)
    ax1 = plt.subplot(gs[0:2, 0:2])
    
    factor_stats = compute_factor_stats(factor, pricing, factor_returns, factor_loadings,
                                        periods=periods, view=view)
                         
    sharpes = pd.DataFrame({'specific': factor_stats['delay_sharpes_specific'], 
                  'total': factor_stats['delay_sharpes_total']})
#     display(sharpes)
    sharpes.plot.bar(ax=ax1)
    ax1.set(xlabel='delay', ylabel='IR')

    ax2a = plt.subplot(gs[0, 2:4])
    delay_cum_rets_total = factor_stats['portfolio_returns_total'][list(range(1, 5))].apply(ep.cum_returns)
    delay_cum_rets_total.plot(ax=ax2a)
    ax2a.set(title='Total returns', ylabel='Cumulative returns')
    
    ax2b = plt.subplot(gs[1, 2:4])
    delay_cum_rets_specific = factor_stats['portfolio_returns_specific'][list(range(1, 5))].apply(ep.cum_returns)
    delay_cum_rets_specific.plot(ax=ax2b)
    ax2b.set(title='Specific returns', ylabel='Cumulative returns')
    
    ax3 = plt.subplot(gs[2:4, 0:2])
    plot_exposures(factor_stats['risk_exposures_portfolio'].reindex(columns=factor_stats['perf_attribution'].columns), 
                   ax=ax3)

    ax4 = plt.subplot(gs[2:4, 2])
    ep.cum_returns_final(factor_stats['perf_attribution']).plot.barh(ax=ax4)
    ax4.set(xlabel='Cumulative returns')

    ax5 = plt.subplot(gs[2:4, 3], sharey=ax4)
    factor_stats['perf_attribution'].apply(ep.annual_volatility).plot.barh(ax=ax5)
    ax5.set(xlabel='Ann. volatility')

    ax6 = plt.subplot(gs[-1, 0:2])
    factor_stats['n_holdings'].plot(color='b', ax=ax6)
    ax6.set_ylabel('# holdings', color='b')
    ax6.tick_params(axis='y', labelcolor='b')
    
    ax62 = ax6.twinx()
    factor_stats['turnover'].plot(color='r', ax=ax62)
    ax62.set_ylabel('turnover', color='r')
    ax62.tick_params(axis='y', labelcolor='r')
    
    ax7 = plt.subplot(gs[-1, 2:4])
    factor_stats['perc_holdings'].plot(ax=ax7)
    ax7.set_ylabel('Holdings Ratio')
    
    gs.tight_layout(fig)
    
    return fig, factor_stats, sharpes

In [ ]:
# Loop through all columns
results = None
for i, col in enumerate(sorted(alphas_view.columns)):
    if i > 0:
        print('')
    print(col)
    
    # Get the factor data
    try:
        data = alphas_view[col]
        data = data[data != 0].dropna()
        fig, factor_stats, sharpes = plot_overview_tear_sheet(data,
                                                     prices,
                                                     factor_returns,
                                                     factor_loadings);
        plt.show()
        
        sharpes.columns = pd.MultiIndex.from_product([[col], sharpes.columns])
        if results is None:
            results = sharpes
        else:
            results = pd.concat([results, sharpes], axis=1)
        
    except Exception as e:
        print('Error: {}'.format(e))
        continue
        
# results